In [1]:
# using Pkg
# Pkg.add("BenchmarkTools")

using Plots
using Base.Threads
using BenchmarkTools

In [2]:
@enum BoundaryType begin
    NoBoundary
    BounceBack
    Dirichlet
    Adiabatic
    Open
end

"""
# Summary
    struct BoundaryCell

## Note
- If no boundary is specified, use [nothing] instead of an empty vector []. The latter will cause an error.

# Fields
- "BoundaryNorth::Vector{Any}": The first element must be of type `::BoundaryType`, and the rest are parameters for that boundary type.
- "BoundarySouth::Vector{Any}": The first element must be of type `::BoundaryType`, and the rest are parameters for that boundary type.
- "BoundaryWest::Vector{Any}": The first element must be of type `::BoundaryType`, and the rest are parameters for that boundary type.
- "BoundaryEast::Vector{Any}": The first element must be of type `::BoundaryType`, and the rest are parameters for that boundary type.
"""
struct SolidCell
    BoundaryNorth::BoundaryType
    BoundarySouth::BoundaryType
    BoundaryWest::BoundaryType
    BoundaryEast::BoundaryType
    BoundaryNorthParameters::Vector{Float64}
    BoundarySouthParameters::Vector{Float64}
    BoundaryWestParameters::Vector{Float64}
    BoundaryEastParameters::Vector{Float64}

    SolidCell() = new(NoBoundary, NoBoundary, NoBoundary, NoBoundary,
                      Vector{Float64}(), Vector{Float64}(), Vector{Float64}(), Vector{Float64}())

    SolidCell(boundaryNorth::BoundaryType, boundarySouth::BoundaryType, boundaryWest::BoundaryType, boundaryEast::BoundaryType) =
        new(boundaryNorth, boundarySouth, boundaryWest, boundaryEast,
            Vector{Float64}(), Vector{Float64}(), Vector{Float64}(), Vector{Float64}())

    SolidCell(boundaryNorth::BoundaryType, boundarySouth::BoundaryType, boundaryWest::BoundaryType, boundaryEast::BoundaryType,
              boundaryNorthParameters::Vector{Float64}, boundarySouthParameters::Vector{Float64},
              boundaryWestParameters::Vector{Float64}, boundaryEastParameters::Vector{Float64}) =
        new(boundaryNorth, boundarySouth, boundaryWest, boundaryEast,
            boundaryNorthParameters, boundarySouthParameters, boundaryWestParameters, boundaryEastParameters)
end

SolidBlock = SolidCell(BounceBack, BounceBack, BounceBack, BounceBack)

SolidCell(BounceBack, BounceBack, BounceBack, BounceBack, Float64[], Float64[], Float64[], Float64[])

In [3]:
function LatticeBoltzmannMethodD2Q9(timeSteps::Integer,
        alpha::Float64, u::Vector{Float64},
        initialConditions::Array{Float64, 3}, boundaryConditions, boundaryMap::Matrix{SolidCell}, sourceMap)

    # Pre-check
    @assert timeSteps > 0
    
    @assert length(u) == 2
    
    @assert size(initialConditions, 2) >= 10
    @assert size(initialConditions, 1) >= 10
    @assert size(initialConditions, 3) == 2
    @assert length(size(initialConditions)) == 3
    
    @assert size(boundaryConditions, 1) == 4
    @assert length(size(boundaryConditions)) == 1
    
    @assert size(initialConditions, 2) == size(boundaryMap, 2)
    @assert size(initialConditions, 1) == size(boundaryMap, 1)
    @assert length(size(boundaryMap)) == 2
    
    @show timeSteps
    M = size(initialConditions, 2) # M is x (column)
    N = size(initialConditions, 1) # N is y (row)
    print("dim = $N x $M, xlim = $M, ylim = $N\n")
    
    # D2Q9 Meta data
    # - f_1 corresonding to speed c(1,0)
    # - f_2 corresonding to speed c(0,1)
    # - f_3 corresonding to speed c(-1,0)
    # - f_4 corresonding to speed c(0,-1)
    # - f_5 corresonding to speed c(1,1)
    # - f_6 corresonding to speed c(-1,1)
    # - f_7 corresonding to speed c(-1,-1)
    # - f_8 corresonding to speed c(1,-1)
    # - f_9 corresonding to speed c(0,0)
    dt = 1.0
    dx = 1.0
    dy = 1.0
    D = 2 + 1
    weight = [1/9, 1/9, 1/9, 1/9, 1/36, 1/36, 1/36, 1/36, 4/9]
    c = [[1, 0], [0, 1], [-1, 0], [0, -1], [1, 1], [-1, 1], [-1, -1], [1, -1], [0, 0]]
    klim = length(weight)
    omega = 1.0 / (((D * alpha) / ((dx*dx) / dt)) + 0.5)
    c_k = [dx / dt, dy / dt]
    c_s_squared_inverse = 3 # Assume dx = dy = dt
    
    
    # Initiate lattice structure
    f = fill(0.0, (N, M, klim)) # density
    g = fill(0.0, (N, M, klim)) # temperature
    @threads for k in 1:klim
        f[:, :, k] = weight[k] .* InitialConditions[:, :, 1]
        g[:, :, k] = weight[k] .* InitialConditions[:, :, 2]
    end    
    # Initiate helper data structure
    temp = fill(0.0, (N, M))
    feq = fill(0.0, (N, M))
    f_updated = similar(f)
    result = zeros(N, M)
    
    # Main loop
    for t in 1:timeSteps
        # Collision Process
        temp = sum(c_k .* u) * c_s_squared_inverse
        feq = sum(f[:, :, j] for j in 1:9)
        @inbounds @threads for k in 1:klim
                    # feq = weight[k]*sum(f[n, m, :])*(1 + temp)
                    # feq = weight[k]*sum(f[n, m, :])*(1 + temp + 0.5*temp*temp - 0.5 * sum(u .* u) * c_s_squared_inverse)
                    f_updated[:, :, k] = f[:, :, k].*(1 - omega) + omega * weight[k] .* feq + dt * weight[k] .* sourceMap[:, :]
        end
        copyto!(f, f_updated)

        # Streaming Process
        f[:, 2:M, 1] = f[:, 1:M-1, 1]
        f[:, 1, 1] .= 0.0

        f[:, 1:M-1, 3] = f[:, 2:M, 3]
        f[:, M, 3] .= 0.0

        f[1:N-1, :, 2] = f[2:N, :, 2]
        f[N, :, 2] .= 0.0

        f[2:N, :, 4] = f[1:N-1, :, 4]
        f[1, :, 4] .= 0.0
        
        f[1:N-1, 1:M-1, 7] = f[2:N, 2:M, 7]
        f[N, :, 7] .= 0.0
        f[:, M, 7] .= 0.0

        f[2:N, 2:M, 5] = f[1:N-1, 1:M-1, 5]
        f[1, :, 5] .= 0.0
        f[:, 1, 5] .= 0.0

        f[2:N, 1:M-1, 6] = f[1:N-1, 2:M, 6]
        f[1, :, 6] .= 0.0
        f[:, M, 6] .= 0.0

        f[1:N-1, 2:M, 8] = f[2:N, 1:M-1, 8]
        f[N, :, 8] .= 0.0
        f[:, 1, 8] .= 0.0
        
        # Boundary conditions, corner points are not important
        @inbounds @threads for m in 2:(M-1)
            f[N, m, :] = MetaBoundaryConditionEvaluation(boundaryConditions[1], [7, 4, 8], f[N, m, :], f[N - 1, m, :])
            f[1, m, :] = MetaBoundaryConditionEvaluation(boundaryConditions[2], [2, 6, 5], f[1, m, :], f[2, m, :])
        end

        @inbounds @threads for n in 2:(N-1)
            f[n, 1, :] = MetaBoundaryConditionEvaluation(boundaryConditions[3], [5, 1, 8], f[n, 1, :], f[n, 2, :])
            f[n, M, :] = MetaBoundaryConditionEvaluation(boundaryConditions[4], [6, 3, 7], f[n, M, :], f[n, M - 1, :])
        end
        
    end
    
    @inbounds @simd for k in 1:klim
        result .+= f[:, :, k]
    end
    
    print("Computation Finished.\n")
    return result
end

function MetaBoundaryConditionEvaluation(boundaryCondition, updateDirection, boundaryNode, cloestNode)
    if typeof(boundaryCondition) == Vector{Any}
        boundaryType = boundaryCondition[1]
    else
        boundaryType = boundaryCondition
    end
    
    weight = [1/9, 1/9, 1/9, 1/9, 1/36, 1/36, 1/36, 1/36, 4/9]
    direction = [3, 4, 1, 2, 7, 8, 5, 6, 0]
    if boundaryType == NoBoundary
    elseif boundaryType == Dirichlet
        for k in updateDirection
            boundaryNode[k] = (weight[k]+weight[direction[k]])*boundaryCondition[2] - boundaryNode[direction[k]]
        end
    elseif boundaryType == Adiabatic
        return cloestNode
    end
    
    return boundaryNode
end

MetaBoundaryConditionEvaluation (generic function with 1 method)

In [4]:
# At Advection-Diffusion Phase, this setup use roughly 0.7s. Retry this after CH5, 6.
M = 1000
N = 1000
t = 10

# z == 1 initial density, z == 2 initial temperature
InitialConditions = zeros(Float64, N, M, 2)
# InitialConditions[6, :, 1] .= 1.0
# InitialConditions[:, 6, 1] .= 1.0
# heatmap(1:20, 1:10, InitialCondition[:,:,1])

# Assign boundary cells to border
BoundaryConditions = [[Dirichlet, 0.0], Adiabatic, [Dirichlet, 1.0], [Dirichlet, 1.0]]

# Add block in region
BoundaryMap        = Matrix{SolidCell}(undef, N, M)
fill!(BoundaryMap, SolidCell())
BoundaryMap[5, 5] = SolidBlock

SourceMap = fill(0.0, N, M);

In [5]:
result = LatticeBoltzmannMethodD2Q9(t, 0.25, [0.2, 0.1], InitialConditions, BoundaryConditions, BoundaryMap, SourceMap);

timeSteps = 10
dim = 1000 x 1000, xlim = 1000, ylim = 1000
Computation Finished.


In [6]:
@benchmark LatticeBoltzmannMethodD2Q9(t, 0.25, [0.0, 0.0], InitialConditions, BoundaryConditions, BoundaryMap, SourceMap) samples=2 evals=2

timeSteps = 10
dim = 1000 x 1000, xlim = 1000, ylim = 1000
Computation Finished.
timeSteps = 10
dim = 1000 x 1000, xlim = 1000, ylim = 1000
Computation Finished.
timeSteps = 10
dim = 1000 x 1000, xlim = 1000, ylim = 1000
Computation Finished.


BenchmarkTools.Trial: 1 sample with 2 evaluations.
 Single result which took 11.849 s (5.03% GC) to evaluate,
 with a memory estimate of 6.47 GiB, over 563548 allocations.